In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_covid()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.15)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (361, 80)
19.75% missing data
Class distribution: (array([0, 1]), array([195, 166]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': [80, 80],
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 4001, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 0.63 sec - Loss 0.100253 - ACC 78.60% - ACC Mean 78.60% - AUC 86.69% - AUC Mean 86.69% - Deter 000
Ite 00050 - 0.11 sec - Loss 0.094004 - ACC 76.94% - ACC Mean 77.79% - AUC 85.43% - AUC Mean 86.57% - Deter 046
Ite 00100 - 0.11 sec - Loss 0.078099 - ACC 77.59% - ACC Mean 76.83% - AUC 86.04% - AUC Mean 86.00% - Deter 096
Ite 00150 - 0.48 sec - Loss 0.054582 - ACC 77.03% - ACC Mean 77.76% - AUC 85.30% - AUC Mean 86.31% - Deter 146
Ite 00200 - 0.11 sec - Loss 0.043222 - ACC 77.00% - ACC Mean 77.84% - AUC 84.69% - AUC Mean 85.50% - Deter 196
Ite 00250 - 0.09 sec - Loss 0.040511 - ACC 77.60% - ACC Mean 77.35% - AUC 85.22% - AUC Mean 85.01% - Deter 246
Ite 00300 - 0.18 sec - Loss 0.039656 - ACC 77.60% - ACC Mean 77.39% - AUC 85.16% - AUC Mean 85.00% - Deter 296
Ite 00350 - 0.12 sec - Loss 0.039176 - ACC 77.86% - ACC Mean 77.46% - AUC 85.45% - AUC Mean 85.15% - Deter 346
Ite 00400 - 0.48 sec - Loss 0.038790 - ACC 77.85% - ACC Mean 77.72% - AUC 85.28% - AUC Mean 85.34% - Deter 396
I

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 77.5% +- 0.0% - auc: 85.27% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 76.65% +- 5.13% - auc: 83.31% +- 4.71%
SFIL RUN 2/10 - acc: 79.21% +- 5.12% - auc: 85.98% +- 4.2%
SFIL RUN 3/10 - acc: 78.06% +- 4.56% - auc: 83.05% +- 6.13%
SFIL RUN 4/10 - acc: 73.16% +- 5.4% - auc: 81.53% +- 5.57%
SFIL RUN 5/10 - acc: 76.82% +- 5.68% - auc: 84.85% +- 2.52%
SFIL RUN 6/10 - acc: 80.87% +- 5.1% - auc: 84.32% +- 4.98%
SFIL RUN 7/10 - acc: 76.26% +- 3.42% - auc: 83.75% +- 3.72%
SFIL RUN 8/10 - acc: 76.0% +- 6.15% - auc: 82.87% +- 7.78%
SFIL RUN 9/10 - acc: 80.34% +- 4.55% - auc: 85.55% +- 4.36%
SFIL RUN 10/10 - acc: 78.02% +- 3.16% - auc: 83.58% +- 3.4%
SFIL GLOBAL - acc: 77.54% +- 2.16% - auc: 83.88% +- 1.27%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 80.02% +- 2.09% - auc: 85.0% +- 0.94%
PFIL RUN 2/10 - acc: 78.09% +- 3.55% - auc: 84.42% +- 2.02%
PFIL RUN 3/10 - acc: 80.94% +- 4.69% - auc: 87.78% +- 3.64%
PFIL RUN 4/10 - acc: 80.01% +- 3.0% - auc: 84.22% +- 1.86%
PFIL RUN 5/10 - acc: 80.21% +- 2.86% - auc: 84.67% +- 4.4%
PFIL RUN 6/10 - acc: 82.31% +- 0.83% - auc: 88.02% +- 1.84%
PFIL RUN 7/10 - acc: 76.04% +- 2.44% - auc: 83.74% +- 2.9%
PFIL RUN 8/10 - acc: 78.29% +- 1.45% - auc: 83.18% +- 1.95%
PFIL RUN 9/10 - acc: 80.32% +- 4.62% - auc: 85.66% +- 4.28%
PFIL RUN 10/10 - acc: 79.63% +- 2.3% - auc: 85.78% +- 4.01%
PFIL GLOBAL - acc: 79.59% +- 1.64% - auc: 85.25% +- 1.52%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 79.19% +- 3.19% - auc: 82.37% +- 3.2%
SPOL RUN 2/10 - acc: 79.25% +- 1.73% - auc: 85.71% +- 1.67%
SPOL RUN 3/10 - acc: 75.84% +- 4.74% - auc: 82.81% +- 3.6%
SPOL RUN 4/10 - acc: 79.23% +- 4.28% - auc: 83.14% +- 4.23%
SPOL RUN 5/10 - acc: 77.49% +- 4.27% - auc: 84.31% +- 3.61%
SPOL RUN 6/10 - acc: 80.09% +- 2.35% - auc: 84.33% +- 4.28%
SPOL RUN 7/10 - acc: 78.99% +- 5.48% - auc: 83.08% +- 5.34%
SPOL RUN 8/10 - acc: 76.61% +- 3.9% - auc: 81.84% +- 2.34%
SPOL RUN 9/10 - acc: 82.16% +- 3.64% - auc: 86.94% +- 4.62%
SPOL RUN 10/10 - acc: 78.89% +- 3.73% - auc: 83.86% +- 4.91%
SPOL GLOBAL - acc: 78.77% +- 1.7% - auc: 83.84% +- 1.48%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 80.94% +- 5.65% - auc: 86.2% +- 3.92%
PPOL RUN 2/10 - acc: 79.46% +- 4.49% - auc: 84.85% +- 3.84%
PPOL RUN 3/10 - acc: 77.9% +- 3.96% - auc: 85.58% +- 2.98%
PPOL RUN 4/10 - acc: 77.73% +- 4.9% - auc: 81.65% +- 3.3%
PPOL RUN 5/10 - acc: 78.49% +- 3.64% - auc: 83.27% +- 4.82%
PPOL RUN 6/10 - acc: 82.3% +- 3.91% - auc: 86.51% +- 2.69%
PPOL RUN 7/10 - acc: 77.76% +- 3.9% - auc: 85.13% +- 1.89%
PPOL RUN 8/10 - acc: 78.53% +- 3.02% - auc: 84.12% +- 3.72%
PPOL RUN 9/10 - acc: 77.26% +- 4.65% - auc: 81.43% +- 3.63%
PPOL RUN 10/10 - acc: 81.46% +- 4.59% - auc: 86.05% +- 3.72%
PPOL GLOBAL - acc: 79.18% +- 1.69% - auc: 84.48% +- 1.74%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is even with SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is even with PFIL
Metric AUC - OUR METHOD is significantly better than PFIL
Metric ACC - OUR METHOD is even with SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is even with PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
